# Observabilité, Evaluation | Langfuse



### Reprenons un simple chat bot

In [ ]:
#Décommenter sur Google Colab
#%pip install langchain langchain-openai langchain_mistralai openai python-dotenv -q
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')


#Décommenter en local
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key= getenv("OPENAI_API_KEY")


from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

langfuse = Langfuse(
  secret_key=getenv("LANGFUSE_SECRET_KEY"),
  public_key=getenv("LANGFUSE_PUBLIC_KEY"),
  host="https://cloud.langfuse.com"
)
langfuse_handler = CallbackHandler()

In [8]:
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Répond en français."), #Ici des instructions pour le modèle, 'à l'insu' de l'utilisateur
    MessagesPlaceholder(variable_name="messages"), # Ici on utilise un placeholder pour la mémoire, qui sera remplacé par l'historique des messages automatiquement par LangChain
    ("human", "{question}"), # Ici la question de l'utilisateur, variable {question} à remplacer sera remplacée par la question de l'utilisateur
])
llm_openai = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0
)
chatbot = prompt | llm_openai
chat_history = ChatMessageHistory()
config = {
    "callbacks": [langfuse_handler],
    "metadata": {
        "langfuse_session_id": "chatbot_session_1",
        "langfuse_user_id": "user_12345",  # Optionnel, pour identifier l'utilisateur
        "langfuse_tags": ["tag-1", "evaluate"] #evaluate
    }
}

question1 = "Qui est le président de la France ?"

response = chatbot.invoke(
    {"question": question1, "messages": chat_history.messages}, 
    config=config)
print(f"Chatbot: {response.content}", flush=True)

chat_history.add_user_message(question1)
chat_history.add_ai_message(response.content)

question2 = "Et du Quebec ?"

response = chatbot.invoke({"question": question2, "messages": chat_history.messages},  config=config)
print(f"Chatbot: {response.content}", flush=True)

Chatbot: En octobre 2023, le président de la France est Emmanuel Macron. Il est en fonction depuis mai 2017.
Chatbot: En octobre 2023, le premier ministre du Québec est François Legault. Il est à la tête du gouvernement depuis octobre 2018.


### Avec un workflow complexe

In [10]:
from langgraph.types import Send
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from operator import add
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel
from langchain_core.runnables import RunnableConfig
from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

langfuse = Langfuse(
  secret_key=getenv("LANGFUSE_SECRET_KEY"),
  public_key=getenv("LANGFUSE_PUBLIC_KEY"),
  host="https://cloud.langfuse.com"
)
langfuse_handler = CallbackHandler()

def setup_config(tags: list[str] = []) -> RunnableConfig : 
    return  {
    "callbacks": [langfuse_handler],
    "metadata": {
        "langfuse_session_id": "langgraph_session",  # ID de session unique pour Langfuse
        "langfuse_user_id": "user_6789",  # Optionnel, pour identifier l'utilisateur
        "langfuse_tags": tags #evaluate
    }
}



class GeneratedTopics(BaseModel):
    topics: list[str]

class StreamingState(TypedDict):
    contraints: str
    topics: list[str]
    jokes: Annotated[list[str], add]
    best_joke: str

class JokeState(TypedDict):
    topic: str
    contraints: str

# NODES
async def generate_topics(state: StreamingState):
    model = ChatOpenAI(model="gpt-4o-mini").with_structured_output(GeneratedTopics)
    response = PromptTemplate.from_template(
        "Generate a list of 3 random one word topics for jokes suitable for theses given constraint. Do not generate the jokes only the topics.\n"
        "Here are the constraints: {input}."
    ).pipe(model).invoke({"input": state["contraints"]}, config=setup_config(["generate_topics"]))
    return {"topics": GeneratedTopics.model_validate(response).topics}
    
async def generate_joke(state: JokeState):
    model = ChatOpenAI(model="gpt-4o-mini")
    response = PromptTemplate.from_template(
        "Generate a short joke about {topic} that is suitable for theses given constraint: {contraints}."
    ).pipe(model).invoke({"topic": state["topic"], "contraints": state["contraints"]}, config=setup_config(["generate_joke"]))
    return {"jokes": [response.content]}

async def select_best_joke(state: StreamingState):
    model = ChatOpenAI(model="gpt-4o-mini")
    response = PromptTemplate.from_template(
        "Here are three jokes, select the most appropriate one for theses given constraint: {contraints}.\n. Your response must be only the selected joke, nothing else.\n Here are the jokes:\n{jokes}\n"
    ).pipe(model).invoke({"jokes": "\n".join(state["jokes"]), "contraints": state["contraints"]}, config=setup_config(["select_best_joke"]))
    return {"best_joke": response.content}

def parallelize(state: StreamingState):
    # Return a list of Send actions to invoke node2 in parallel
    return [Send("generate_joke", {"topic": topic, "contraints": state["contraints"]}) for topic in state["topics"]]


# GRAPH
graph_builder = StateGraph(StreamingState)

graph_builder.add_node(generate_topics)
graph_builder.add_node(generate_joke)
graph_builder.add_node(select_best_joke)

graph_builder.add_edge(START, "generate_topics")
graph_builder.add_conditional_edges("generate_topics",parallelize, ["generate_joke"])
graph_builder.add_edge("generate_joke", "select_best_joke")
joke_graph = graph_builder.compile()

output = await joke_graph.ainvoke({"contraints": "An inoffensive joke about animals, suitable for a young kid",})
output

No 'langfuse_public_key' passed to decorated function, but multiple langfuse clients are instantiated in current process. Skipping tracing for this function to avoid cross-project leakage.


{'contraints': 'An inoffensive joke about animals, suitable for a young kid',
 'topics': ['Elephant', 'Penguin', 'Monkey'],
 'jokes': ["Why don't elephants use computers?\n\nBecause they're afraid of the mouse!",
  "Why don't penguins like talking to strangers at parties?  \n\nBecause they find it hard to break the ice! 🐧😄",
  'What do you call a monkey in a tree?  \nA hot monkey!'],
 'best_joke': "Why don't penguins like talking to strangers at parties?  \n\nBecause they find it hard to break the ice! 🐧😄"}